# Normalization Testing

Normalization simple means that we detect the percentile of an observation, and we can translate it to the same percentile in another industry.

The values associated to each percentile change in time, and per industry. However the percentile associated to each observation we assume as being translatable.

For MVP, the use of percentiles appear as enough, because we see some stability in relation to the global percentiles. SO just calculating the percentile, global and internal, will be considered enough for any audit, and inmediate translation can be considered.

**Future Improvements:**
Future improvements that can be made for this analogy is the inverse translation from a percentile to an actual value.
This actual value have a trend in time, n relation with the percentile. For example we have seen that the deduc to income ratio have increase in industry NAICS 23. So, a percentile 75% today is a higher ratio than a percentile 75% 10 years ago. However, this trend is pretty linear, and could be added in the reconstruction of actual historical values if needed.

**Testing**
The testing we need to do is about the calculation of percentiles. Particularly determine the `method` we want to apply (linear, dense, min, etc.)

In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
%aimport audit_functions

In [110]:
import pandas as pd
import numpy as np
from typing import List, Dict, Optional

from dstools.mlutils.corp_tax_audit_unsupervised import (add_abs_diffs, compare_dataframes)



In [4]:
%store -r q_industries
%store -r q_audit_data_combined

In [5]:
# Comparing different methods for percentiling
a = [0, 0, 0, 1, 1, 1, 1.5, 2, 2]
b = [1, 2, 3, 4, 5, 6, 7,   8, 9]
c = [2, 2, 2, 3, 3, 3, 4,   4, 4]
a1 = pd.Series(a)
b1 = pd.Series(b)
c1 = pd.Series(c)

In [6]:
# I like this method, because it will map duplicate percentiles to the average of them, so it will not go to extremes. 
# Also, it will go to the same percentiles for the observations in the ranking that are not duplicated.
method = 'average'
average = pd.concat([a1, b1, c1, a1.rank(method=method, pct=True), 
                   b1.rank(method=method, pct=True), 
                   c1.rank(method=method, pct=True)],
                  axis = 1)
print(method)
average

average


,0,1,2,3,4,5
0,0.0,1,2,0.222222,0.111111,0.222222
1,0.0,2,2,0.222222,0.222222,0.222222
2,0.0,3,2,0.222222,0.333333,0.222222
3,1.0,4,3,0.555556,0.444444,0.555556
4,1.0,5,3,0.555556,0.555556,0.555556
5,1.0,6,3,0.555556,0.666667,0.555556
6,1.5,7,4,0.777778,0.777778,0.888889
7,2.0,8,4,0.944444,0.888889,0.888889
8,2.0,9,4,0.944444,1.000000,0.888889


In [7]:
method = 'dense'
dense = pd.concat([a1, b1, c1, a1.rank(method=method, pct=True), 
                   b1.rank(method=method, pct=True), 
                   c1.rank(method=method, pct=True)],
                  axis = 1)
print(method)
dense

dense


,0,1,2,3,4,5
0,0.0,1,2,0.25,0.111111,0.333333
1,0.0,2,2,0.25,0.222222,0.333333
2,0.0,3,2,0.25,0.333333,0.333333
3,1.0,4,3,0.50,0.444444,0.666667
4,1.0,5,3,0.50,0.555556,0.666667
5,1.0,6,3,0.50,0.666667,0.666667
6,1.5,7,4,0.75,0.777778,1.000000
7,2.0,8,4,1.00,0.888889,1.000000
8,2.0,9,4,1.00,1.000000,1.000000


In [8]:
method = 'min'
min_meth = pd.concat([a1, b1, c1, a1.rank(method=method, pct=True), 
                   b1.rank(method=method, pct=True), 
                   c1.rank(method=method, pct=True)],
                  axis = 1)
print(method)
min_meth

min


,0,1,2,3,4,5
0,0.0,1,2,0.111111,0.111111,0.111111
1,0.0,2,2,0.111111,0.222222,0.111111
2,0.0,3,2,0.111111,0.333333,0.111111
3,1.0,4,3,0.444444,0.444444,0.444444
4,1.0,5,3,0.444444,0.555556,0.444444
5,1.0,6,3,0.444444,0.666667,0.444444
6,1.5,7,4,0.777778,0.777778,0.777778
7,2.0,8,4,0.888889,0.888889,0.777778
8,2.0,9,4,0.888889,1.000000,0.777778


In [9]:
method = 'first'
first_meth = pd.concat([a1, b1, c1, a1.rank(method=method, pct=True), 
                   b1.rank(method=method, pct=True), 
                   c1.rank(method=method, pct=True)],
                  axis = 1)
print(method)
first_meth

first


,0,1,2,3,4,5
0,0.0,1,2,0.111111,0.111111,0.111111
1,0.0,2,2,0.222222,0.222222,0.222222
2,0.0,3,2,0.333333,0.333333,0.333333
3,1.0,4,3,0.444444,0.444444,0.444444
4,1.0,5,3,0.555556,0.555556,0.555556
5,1.0,6,3,0.666667,0.666667,0.666667
6,1.5,7,4,0.777778,0.777778,0.777778
7,2.0,8,4,0.888889,0.888889,0.888889
8,2.0,9,4,1.000000,1.000000,1.000000


In [10]:
q_industries.head()

,naics_code,business_id,bus_loc_id,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,sumsum_deduc,sumsum_taxable,sumsum_paid,eff_tax_rate%,deduc_to_income_ratio,naics2,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind
date,,,,,,,,,,,,,,,,,,
2001Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,16193.58,0.00,16193.58,67.20,0.414979,0.000000,22,0.728517,0.909091,0.000420,0.333333
2001Q2,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,158968.92,0.00,158968.92,659.72,0.414999,0.000000,22,0.822274,1.000000,0.000425,0.333333
2001Q3,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,95335.05,20497.55,74837.50,310.58,0.325777,0.215005,22,0.630379,0.909091,0.617837,0.750000
2001Q4,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,36138.27,36138.27,0.00,0.00,0.000000,1.000000,22,0.000097,0.090909,1.000000,1.000000
2002Q1,221122,109874,110874,ISLAND UTILITY COMPANY,1,Q,10,7334.92,0.00,7334.92,30.44,0.415001,0.000000,22,0.906278,1.000000,0.000449,0.333333


In [11]:
test_data = {
    'date': ['2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4'],
    'business_id': ['A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C'],
    'ded_test': [0, 0, 0, 0, 0.25, 0.5, 1, 0, 1, 0.2, 0, 0]
}

df = pd.DataFrame(test_data)
df


,date,business_id,ded_test
0,2020Q1,A,0.00
1,2020Q2,A,0.00
2,2020Q3,A,0.00
3,2020Q4,A,0.00
4,2020Q1,B,0.25
5,2020Q2,B,0.50
6,2020Q3,B,1.00
7,2020Q4,B,0.00
8,2020Q1,C,1.00
9,2020Q2,C,0.20


In [12]:
df[df['date']=='2020Q1']['ded_test'].rank(method = 'average', pct=True)

0    0.333333
4    0.666667
8    1.000000
Name: ded_test, dtype: float64

In [13]:
df[df['date']=='2020Q2']['ded_test'].rank(method = 'average', pct=True)

1    0.333333
5    1.000000
9    0.666667
Name: ded_test, dtype: float64

In [14]:
df[df['date']=='2020Q3']['ded_test'].rank(method = 'average', pct=True)

2     0.5
6     1.0
10    0.5
Name: ded_test, dtype: float64

In [15]:
df[df['date']=='2020Q4']['ded_test'].rank(method = 'average', pct=True)

3     0.666667
7     0.666667
11    0.666667
Name: ded_test, dtype: float64

In [16]:
test_data = {
    'date': ['2019Q4','2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
            '2021Q1', '2021Q1'],
    'business_id': ['A','A', 'A', 'A', 'A', 'B', 'B', 'B', 'B', 'C', 'C', 'C', 'C','B','C'],
    'ded_test': [1, 0, 0, 0, 0, 0.25, 0.5, 1, 0, 1, 0.2, 0, 0, 0.8, 0.3]
}

df = pd.DataFrame(test_data)
df

,date,business_id,ded_test
0,2019Q4,A,1.00
1,2020Q1,A,0.00
2,2020Q2,A,0.00
3,2020Q3,A,0.00
4,2020Q4,A,0.00
5,2020Q1,B,0.25
6,2020Q2,B,0.50
7,2020Q3,B,1.00
8,2020Q4,B,0.00
9,2020Q1,C,1.00


In [17]:
df[df['date']=='2019Q4']['ded_test'].rank(method = 'average', pct=True)

0    1.0
Name: ded_test, dtype: float64

In [18]:
df[df['date']=='2021Q1']['ded_test'].rank(method = 'average', pct=True)

13    1.0
14    0.5
Name: ded_test, dtype: float64

In [19]:
DATA = {
    'date': ['2019Q4','2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4','2021Q1', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1'],
    'business_id': ['A','A', 'A', 'A', 'A', 'B', 'B', 'B', 'B','B', 'C', 'C', 'C', 'C','C'],
    'ded_test': [1, 0, 0, 0, 0, 0.25, 0.5, 1, 0, 0.8, 1, 0.2, 0, 0, 0.3]
}

DATA = pd.DataFrame(DATA)
DATA['date'] = pd.PeriodIndex(DATA['date'], freq='Q')
DATA_no_index = DATA.copy()
DATA.set_index('date', inplace=True)
DATA

,business_id,ded_test
date,,
2019Q4,A,1.00
2020Q1,A,0.00
2020Q2,A,0.00
2020Q3,A,0.00
2020Q4,A,0.00
2020Q1,B,0.25
2020Q2,B,0.50
2020Q3,B,1.00
2020Q4,B,0.00


In [20]:
DATA2 = DATA.copy()
DATA2['feat2'] = DATA['ded_test']*2
DATA2

,business_id,ded_test,feat2
date,,,
2019Q4,A,1.00,2.0
2020Q1,A,0.00,0.0
2020Q2,A,0.00,0.0
2020Q3,A,0.00,0.0
2020Q4,A,0.00,0.0
2020Q1,B,0.25,0.5
2020Q2,B,0.50,1.0
2020Q3,B,1.00,2.0
2020Q4,B,0.00,0.0


In [21]:
cols = ['a', 'b']
new_names = None
if new_names is None:
    new_names = [None] * len(cols)
new_names[0] is None
    

True

# Delta variables

In [22]:
def add_abs_diffs_local(X: pd.DataFrame, abs_diff_cols: List[str], 
                  reference_cols: List[str] = ['naics_code', 'business_id', 'bus_loc_id']
                  ) -> pd.DataFrame:

    """
    Add absolute differences (1Q and 4Q differences) for selected variables. 

    Args:
        X (DataFrame): A DataFrame that contains the required columns (reference_cols and abs_diff_cols).
                       It should also have a unique index, which is a time index.
        abs_diff_cols (list): A list of columns to which the absolute differences will be applied.
        reference_cols (list): A list of reference columns used for grouping and indexing. Default is ['naics2',
                               'business_id', 'bus_loc_id'].

    Returns:
        DataFrame: The DataFrame with added absolute differences columns.

    Notes:
        - For deductions, since we already have relative columns, only the absolute difference is calculated for all features.
        - The function need the index to be sorted, so the return dataframe will have an index sorted accordingly, which can be
            different than the original sort.
        - The newly added columns in the output DataFrame (`X_final`) will have the following naming convention:
          - Columns for 1Q differences: "1Delta_abs_{column_name}"
          - Columns for 4Q differences: "4Delta_abs_{column_name}"

    @TODO:
        - Make the function work more extensively for other cases as well.
        - Add a parameter for specifying the period.

    """
    
    
    pd.set_option('mode.use_inf_as_na', True)
    
    # Sorting index - Each group is sorted by date
    X_copy = X.copy()
    X_copy.reset_index(names='date', inplace=True) 
    X_copy.sort_values(by= reference_cols + ['date'])
    X_copy.set_index('date')
    
    X_copy = X_copy.set_index(keys=reference_cols, append=True).copy()

    
    # ABSOLUTE DIFFERENCES
    X_diff_abs = X_copy[abs_diff_cols].copy()

    #take out of the index the reference cols, which are level=[1,2,3]
    index_level_to_join=list(range(1,1+len(reference_cols))) 
    
    # Differences with the previous period
    X_diff_abs_1 = X_diff_abs.groupby(level=index_level_to_join).diff(periods = 1).fillna(0)
    X_diff_abs_1 = X_diff_abs_1.add_prefix("1Delta_abs_")

    X_diff_abs_4 = X_diff_abs.groupby(level=index_level_to_join).diff(periods = 4).fillna(0)
    X_diff_abs_4 = X_diff_abs_4.add_prefix("4Delta_abs_")
   
    X_final = pd.concat([X_copy, X_diff_abs_1, X_diff_abs_4], axis=1) 
    
    # Decided to reset index to just the time series, because it is easy to do the concat that way
    X_final.reset_index(level=index_level_to_join, inplace=True)
    
    
    return X_final

In [26]:
q_industries = add_abs_diffs(X=q_industries, abs_diff_cols=['eff_tax_rate%','deduc_to_income_ratio'], 
                  reference_cols=['naics2', 'business_id', 'bus_loc_id']
             )


In [28]:
q_industries.head()

,naics2,business_id,bus_loc_id,date,naics_code,business_legal_name,maxNumLoc,tax_period_cd,obl_type_id,sumsum_gross,...,eff_tax_rate%,deduc_to_income_ratio,eff_tax%_perc_glob,eff_tax%_perc_ind,deduc2income_perc_glob,deduc2income_perc_ind,1Delta_abs_eff_tax_rate%,1Delta_abs_deduc_to_income_ratio,4Delta_abs_eff_tax_rate%,4Delta_abs_deduc_to_income_ratio
0,22,109874,110874,2001Q1,221122,ISLAND UTILITY COMPANY,1,Q,10,16193.58,...,0.414979,0.000000,0.728517,0.909091,0.000420,0.333333,0.000000,0.000000,0.000000,0.0
1,22,109874,110874,2001Q2,221122,ISLAND UTILITY COMPANY,1,Q,10,158968.92,...,0.414999,0.000000,0.822274,1.000000,0.000425,0.333333,0.000020,0.000000,0.000000,0.0
2,22,109874,110874,2001Q3,221122,ISLAND UTILITY COMPANY,1,Q,10,95335.05,...,0.325777,0.215005,0.630379,0.909091,0.617837,0.750000,-0.089222,0.215005,0.000000,0.0
3,22,109874,110874,2001Q4,221122,ISLAND UTILITY COMPANY,1,Q,10,36138.27,...,0.000000,1.000000,0.000097,0.090909,1.000000,1.000000,-0.325777,0.784995,0.000000,0.0
4,22,109874,110874,2002Q1,221122,ISLAND UTILITY COMPANY,1,Q,10,7334.92,...,0.415001,0.000000,0.906278,1.000000,0.000449,0.333333,0.415001,-1.000000,0.000022,0.0


In [111]:
# Testing
data = {
    'date': ['2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2020Q1', '2020Q2', '2020Q3', '2020Q4',
             '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1',
             '2019Q4', '2020Q2', '2020Q3', '2021Q1'],
    'naics2': ['52', '52', '52', '52', '52', '52', '52', '52', '52', '52',
               '52', '52', '52', '52', '33', '33', '33', '33', '33', '33',
               '34', '34', '34', '34'],
    'business_id': ['ZA', 'ZA', 'ZA', 'ZA', 'ZA', 'ZA', 
                    'B', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 
                    'AC', 'AC', 'AC', 'AC', 'AC', 'AC',
                    'AE', 'AE', 'AE', 'F'],
    'location_id': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
                    3, 3, 3, 3, 4, 4, 4, 4, 4, 4,
                    5, 5, 5, 6],
    'feature1': [0, 0.1, 0, 0.2, 0.5, 1, 1, 0.2, 0.3, 0.2,
                 0.1, 0.9, 0.8, 0.3, 0.5, 2, 3, 4, 5, 6,
                 1, 3, 4, 1],
    'feature2': [0, 1, 2, 3, 4,4.5, 5, 4, 3, 2,
                 1, 1, 1, 1, 1, 2, 3, 4, 5, 6,
                 4, 3, 1, 1]
}

DF = pd.DataFrame(data)
DF.set_index('date', inplace=True)



# Define the expected output DataFrame


# # Call the add_abs_diffs function
# result = add_abs_diffs(X=DF, abs_diff_cols=['feature1', 'feature2'], reference_cols=['naics2', 'business_id', 'location_id'])

# # Compare the actual output with the expected output
# #pd.testing.assert_frame_equal(result, expected_df)


# print(result)

In [112]:
# Sorting index - Each group is sorted by date
X=DF
abs_diff_cols=['feature1', 'feature2']
reference_cols=['naics2', 'business_id', 'location_id']

X_copy = X.copy()
index_name = X_copy.index.name
X_copy.reset_index(inplace=True) 
sort_list = reference_cols + [index_name]
X_copy.sort_values(by= sort_list, inplace=True)
index_cols = [index_name] + reference_cols
X_copy.set_index(keys=index_cols, inplace=True)

    
X_copy

,,,,feature1,feature2
date,naics2,business_id,location_id,,
2019Q4,33,AC,4,0.5,1.0
2020Q1,33,AC,4,2.0,2.0
2020Q2,33,AC,4,3.0,3.0
2020Q3,33,AC,4,4.0,4.0
2020Q4,33,AC,4,5.0,5.0
2021Q1,33,AC,4,6.0,6.0
2019Q4,34,AE,5,1.0,4.0
2020Q2,34,AE,5,3.0,3.0
2020Q3,34,AE,5,4.0,1.0


In [113]:
# ABSOLUTE DIFFERENCES
X_diff_abs = X_copy[abs_diff_cols].copy()
X_diff_abs

#take out of the index the reference cols, which are level=[1,2,3]
index_level_to_join=list(range(1,1+len(reference_cols))) 
index_level_to_join

# Differences with the previous period
X_diff_abs_1 = X_diff_abs.groupby(level=index_level_to_join).diff(periods = 1).fillna(0)
X_diff_abs_1
X_diff_abs_1 = X_diff_abs_1.add_prefix("1Delta_abs_")
X_diff_abs_1

X_diff_abs_4 = X_diff_abs.groupby(level=index_level_to_join).diff(periods = 4).fillna(0)
X_diff_abs_4 = X_diff_abs_4.add_prefix("4Delta_abs_")
X_diff_abs_4

X_final = pd.concat([X_copy, X_diff_abs_1, X_diff_abs_4], axis=1) 
X_final

# Decided to reset index to just the time series, because it is easy to do the concat that way
X_final.reset_index(level=index_level_to_join, inplace=True)
X_final

,naics2,business_id,location_id,feature1,feature2,1Delta_abs_feature1,1Delta_abs_feature2,4Delta_abs_feature1,4Delta_abs_feature2
date,,,,,,,,,
2019Q4,33,AC,4,0.5,1.0,0.0,0.0,0.0,0.0
2020Q1,33,AC,4,2.0,2.0,1.5,1.0,0.0,0.0
2020Q2,33,AC,4,3.0,3.0,1.0,1.0,0.0,0.0
2020Q3,33,AC,4,4.0,4.0,1.0,1.0,0.0,0.0
2020Q4,33,AC,4,5.0,5.0,1.0,1.0,4.5,4.0
2021Q1,33,AC,4,6.0,6.0,1.0,1.0,4.0,4.0
2019Q4,34,AE,5,1.0,4.0,0.0,0.0,0.0,0.0
2020Q2,34,AE,5,3.0,3.0,2.0,-1.0,0.0,0.0
2020Q3,34,AE,5,4.0,1.0,1.0,-2.0,0.0,0.0


In [ ]:
X_final = 

In [128]:

expected_data = {
    'date': ['2019Q4', '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2019Q4', '2020Q2', '2020Q3', '2021Q1', 
             '2020Q1', '2020Q2', '2020Q3', '2020Q4', '2020Q2', '2020Q3', '2020Q4', '2021Q1', '2019Q4', '2020Q1', 
             '2020Q2', '2020Q3', '2020Q4', '2021Q1'],
    'naics2': ['33', '33', '33', '33', '33', '33', '34', '34', '34', '34', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52', '52'],
    'business_id': ['AC', 'AC', 'AC', 'AC', 'AC', 'AC', 
                    'AE', 'AE', 'AE', 
                    'F', 'B', 'B', 'B', 'B', 'B', 'B', 'B', 
                    'B', 'ZA', 'ZA', 'ZA', 'ZA', 'ZA', 'ZA'],
    'location_id': [4, 4, 4, 4, 4, 4, 5, 5, 5, 6, 2, 2, 2, 2, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1],
    'feature1': [0.5, 2.0, 3.0, 4.0, 5.0, 6.0, 1.0, 3.0, 4.0, 1.0, 1.0, 0.2, 0.3, 0.2, 0.1, 0.9, 0.8, 0.3, 0.0, 0.1, 
                 0.0, 0.2, 0.5, 1.0],
    'feature2': [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 4.0, 3.0, 1.0, 1.0, 5.0, 4.0, 3.0, 2.0, 1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 
                 2.0, 3.0, 4.0, 4.5],
    '1Delta_abs_feature1': [0.0, 1.5, 1.0, 1.0, 1.0, 1.0, 0.0, 2.0, 1.0, 0.0, 
                            0.0, -0.8, 0.1, -0.1, 0.0, 0.8, -0.1, -0.5, 
                            0.0, 0.1, -0.1, 0.2, 0.3, 0.5],
    '1Delta_abs_feature2': [0.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, -1.0, -2.0, 0.0, 0.0, -1.0, -1.0, -1.0, 0.0, 0.0, 0.0, 
                            0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 0.5],
    '4Delta_abs_feature1': [0.0, 0.0, 0.0, 0.0, 4.5, 4.0, 
                            0.0, 0.0, 0.0, 0.0, 
                            0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                            0.0, 0.0, 0.0, 0.0, 0.5, 0.9],
    '4Delta_abs_feature2': [0.0, 0.0, 0.0, 0.0, 4.0, 4.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 
                            0.0, 0.0, 0.0, 0.0, 4.0, 3.5]
}

df = pd.DataFrame(expected_data)

print(df)

DF_EXPECTED = pd.DataFrame(expected_data)
DF_EXPECTED.set_index('date', inplace=True)
DF_EXPECTED == X_final


      date naics2 business_id  location_id  feature1  feature2  \
0   2019Q4     33          AC            4       0.5       1.0   
1   2020Q1     33          AC            4       2.0       2.0   
2   2020Q2     33          AC            4       3.0       3.0   
3   2020Q3     33          AC            4       4.0       4.0   
4   2020Q4     33          AC            4       5.0       5.0   
5   2021Q1     33          AC            4       6.0       6.0   
6   2019Q4     34          AE            5       1.0       4.0   
7   2020Q2     34          AE            5       3.0       3.0   
8   2020Q3     34          AE            5       4.0       1.0   
9   2021Q1     34           F            6       1.0       1.0   
10  2020Q1     52           B            2       1.0       5.0   
11  2020Q2     52           B            2       0.2       4.0   
12  2020Q3     52           B            2       0.3       3.0   
13  2020Q4     52           B            2       0.2       2.0   
14  2020Q2

,naics2,business_id,location_id,feature1,feature2,1Delta_abs_feature1,1Delta_abs_feature2,4Delta_abs_feature1,4Delta_abs_feature2
date,,,,,,,,,
2019Q4,True,True,True,True,True,True,True,True,True
2020Q1,True,True,True,True,True,True,True,True,True
2020Q2,True,True,True,True,True,True,True,True,True
2020Q3,True,True,True,True,True,True,True,True,True
2020Q4,True,True,True,True,True,True,True,True,True
2021Q1,True,True,True,True,True,True,True,True,True
2019Q4,True,True,True,True,True,True,True,True,True
2020Q2,True,True,True,True,True,True,True,True,True
2020Q3,True,True,True,True,True,True,True,True,True


In [132]:
import pandas as pd
from pandas.testing import assert_frame_equal

def compare_dataframes(df1, df2, rtol=1e-05, atol=1e-08):
    """
    Compare two dataframes and assert that they are equal or close in values.

    Parameters:
    - df1 (pandas.DataFrame): First dataframe for comparison.
    - df2 (pandas.DataFrame): Second dataframe for comparison.
    - rtol (float): Relative tolerance for numerical comparison. Default is 1e-05.
    - atol (float): Absolute tolerance for numerical comparison. Default is 1e-08.

    Returns:
    None
    """
    # Check if the dataframes have the same shape
    if df1.shape != df2.shape:
        print("Dataframes have different shapes.")
        return

    # Assert that the dataframes are equal or close
    try:
        assert_frame_equal(df1, df2, rtol=rtol, atol=atol)
        print("Dataframes are equal or close.")
    except AssertionError as e:
        print("Dataframes are not equal or close.")
        print(str(e))

# # Example usage
# df1 = pd.DataFrame({'A': [1, 2, 3], 'B': [4, 5, 6]})
# df2 = pd.DataFrame({'A': [1.0001, 2.0002, 3.0003], 'B': [4.0004, 5.0005, 6.0006]})

# compare_dataframes(df1, df2, rtol=1e-04, atol=1e-06)


# To copare dataframes
compare_dataframes(DF_EXPECTED, X_final, rtol=1e-04, atol=1e-06)



Dataframes are equal or close.


In [133]:
X_final

,naics2,business_id,location_id,feature1,feature2,1Delta_abs_feature1,1Delta_abs_feature2,4Delta_abs_feature1,4Delta_abs_feature2
date,,,,,,,,,
2019Q4,33,AC,4,0.5,1.0,0.0,0.0,0.0,0.0
2020Q1,33,AC,4,2.0,2.0,1.5,1.0,0.0,0.0
2020Q2,33,AC,4,3.0,3.0,1.0,1.0,0.0,0.0
2020Q3,33,AC,4,4.0,4.0,1.0,1.0,0.0,0.0
2020Q4,33,AC,4,5.0,5.0,1.0,1.0,4.5,4.0
2021Q1,33,AC,4,6.0,6.0,1.0,1.0,4.0,4.0
2019Q4,34,AE,5,1.0,4.0,0.0,0.0,0.0,0.0
2020Q2,34,AE,5,3.0,3.0,2.0,-1.0,0.0,0.0
2020Q3,34,AE,5,4.0,1.0,1.0,-2.0,0.0,0.0


In [131]:
# CAN BE USED IN THE FUTURE TO TRANSLATE TO REAL VALUES, AFTER HAVING A MATRIX TRANSFORMATION


#np.searchsorted(tables['eff_tax_rate%'].index, audits_normalized['eff_tax%_perc_ind'])


# # Audit Normalization function
# # Takes the value of the feature, and its industry
# audits_normalized = q_audit_data_combined[['naics2','eff_tax%_perc_ind','deduc2income_perc_ind']].reset_index(drop=True).copy() 

# # Transform to use in industry naics0
# naics_to = '23'
# # Tax rate
# lookup_positions = np.searchsorted(tables['eff_tax_rate%'].index, audits_normalized['eff_tax%_perc_ind']) 
# audits_transformed1 = tables['eff_tax_rate%'].iloc[lookup_positions][[naics_to]].reset_index(drop=True)
# audits_transformed1.columns=['eff_tax_rate%']
# # Deduc to income ratio
# feature2 = 'deduc_to_income_ratio'
# lookup_positions = np.searchsorted(tables[feature2].index, audits_normalized['deduc2income_perc_ind']) 
# audits_transformed2 = tables[feature2].iloc[lookup_positions][[naics_to]].reset_index(drop=True)
# audits_transformed2.columns = [feature2]

# print(f"Transformed values for industry NAICS: {naics_to}")
# audits_transformed = pd.concat([audits_transformed1,audits_transformed2], axis=1 )
# audits_transformed.hvplot(kind='scatter',
#                          title = 'Normalized audits transformed to naics 23')
